In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install langchain sentence-transformers faiss-cpu transformers torch gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
from transformers import pipeline
from pathlib import Path
import gradio as gr

In [5]:
# Set up paths
VECTOR_STORE_DIR = Path('/content/drive/MyDrive/CrediTrust_Project/vector_store')
FAISS_INDEX_PATH = VECTOR_STORE_DIR / 'faiss_index.bin'
METADATA_PATH = VECTOR_STORE_DIR / 'metadata.pkl'
CHUNKS_PATH = VECTOR_STORE_DIR / 'chunks.pkl'

In [6]:
# Load vector store and metadata
index = faiss.read_index(str(FAISS_INDEX_PATH))
with open(METADATA_PATH, 'rb') as f:
    metadata = pickle.load(f)
with open(CHUNKS_PATH, 'rb') as f:
    chunks = pickle.load(f)

In [8]:
# Load models
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cpu')
generator = pipeline('text-generation', model='distilgpt2', device=0)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [9]:
# Step 2: RAG pipeline
def rag_pipeline(query, k=5):
    start_time = time.time()

    # Encode query
    query_embedding = model.encode([query], show_progress_bar=False)

    # Retrieve top-k chunks, ensuring valid indices
    k = min(k, index.ntotal)  # Don't exceed index size
    distances, indices = index.search(np.array(query_embedding, dtype=np.float32), k)
    retrieved_chunks = [chunks[idx] for idx in indices[0] if idx < len(chunks)]
    retrieved_metadata = [metadata[idx] for idx in indices[0] if idx < len(metadata)]

    # Create context
    context = "\n".join([f"[{meta['product']}]: {chunk}" for chunk, meta in zip(retrieved_chunks, retrieved_metadata)])

    # Generate answer
    prompt = f"Question: {query}\nContext: {context}\nAnswer concisely:"
    response = generator(prompt, max_new_tokens=100, num_return_sequences=1, truncation=True)[0]['generated_text']
    answer = response.split("Answer concisely:")[-1].strip()

     # Format sources for display
    sources = "\n\n".join([f"Product: {meta['product']}\nComplaint ID: {meta['complaint_id']}\nChunk ID: {meta['chunk_id']}\nText: {chunk}"
                           for chunk, meta in zip(retrieved_chunks, retrieved_metadata)])

    response_time = time.time() - start_time
    return answer, retrieved_chunks, retrieved_metadata, response_time, sources

In [10]:
# Gradio interface
def chatbot_interface(query):
    answer, sources = rag_pipeline(query)
    return answer, sources
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(label="Enter your question", placeholder="e.g., Why are people unhappy with BNPL?"),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Source Chunks and Metadata")
    ],
    title="CrediTrust Complaint Chatbot",
    description="Ask questions about customer complaints for Credit card, Personal loan, BNPL, Savings account, or Money transfers."
)

In [11]:
# Launch interface
iface.launch(share=True)  # Creates a public URL in Colab

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://510b094277ae22ed93.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
